# fewshot提示模板

In [2]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

C:\Users\freestone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [53]:
examples = [
    {"input": "高", "output": "矮"},
    {"input": "胖", "output": "瘦"},
    {"input": "精力充沛", "output": "萎靡不振"},
    {"input": "快乐", "output": "伤心"},
    {"input": "黑", "output": "白"},
]


我们像构造提示词模板对象一样，构造一个普通的 PromptTemplate 对象。

In [59]:
example_prompt=PromptTemplate(input_variables=["input","output"],
    template="""
词语:  {input}\n
反义词:  {output}\n
""")



In [60]:
example_prompt.format(**examples[0])

'\n词语:  高\n\n反义词:  矮\n\n'

调用 format 方法，填入 input 和 output 参数。 当你写`example_prompt.format(**examples[0])`时，`**examples[0]`会将第一个字典的键值对解开，然后作为关键字参数传递给format方法。这等价于example_prompt.format(input="高", output="矮")。

In [96]:
few_shot_prompt = FewShotPromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
  example_separator="\n",
  prefix="我们来玩个反义词游戏，我说词语，你说它的反义词\n",
  suffix="现在轮到你了，词语: {input}\n反义词: ",
  input_variables=["input"],
)
few_shot_prompt.format(input="好")


'我们来玩个反义词游戏，我说词语，你说它的反义词\n\n\n词语:  高\n\n反义词:  矮\n\n\n\n词语:  胖\n\n反义词:  瘦\n\n\n\n词语:  精力充沛\n\n反义词:  萎靡不振\n\n\n\n词语:  快乐\n\n反义词:  伤心\n\n\n\n词语:  黑\n\n反义词:  白\n\n\n现在轮到你了，词语: 好\n反义词: '

我们开始实例化一个 FewShotPromptTemplate 对象，引入 FewShotPromptTemplate 类后，直接使用传参函数式调用。值得注意的是 example单词开始的几个参数。
- 参数 `examples` 是一个字典列表，其中每个字典包含两个键值对：{"input": "高", "output": "矮"}。
- 参数 `example_prompt` 是一个PromptTemplate 对象，不是一个提示词字符串，而是 PromptTemplate 类实例化的对象。
- 参数 `example_separator` 是例子之间使用什么分割符号： \n 代表例子与例子之间是一个空行。


In [91]:
no_suffix_few_shot_prompt = FewShotPromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
  example_separator="\n",
  prefix="我们来玩个反义词游戏，我说词语，你说它的反义词\n",
  input_variables=["input"],
)
no_suffix_few_shot_prompt.format(input="好")


KeyError: 'suffix'

上面的代码我去掉了： 后缀 suffix="词语: {input}\n反义词: " ， 出现报错，说明这个参数是不能省略的。  因为input_variables=["input"], 指定的输入是input变量。只有在suffix 参数中显示了这个变量存放的位置，以及告知 FewShotPromptTemplate 类，实例化的时候，需要填充 input变量 到模板里面。

## 在Chain中使用

In [79]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [86]:
chain = LLMChain(llm=OpenAI(openai_api_key=""),
 prompt=few_shot_prompt )

In [87]:
chain.run("冷")

' 热'

在这段代码中，我们首先实例化了一个LLMChain对象。这个对象是LangChain库中的一个核心组件，可以理解为一个执行链，它将各个步骤连接在一起，形成一个完整的运行流程。LLMChain对象在实例化时需要两个关键参数：一个是llm，这里我们使用了OpenAI提供的大型语言模型；另一个是prompt，这里我们传入的是我们刚刚创建的few_shot_prompt对象。

##  基于长度选择的实例选择器：LengthBasedExampleSelector

In [98]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [100]:
example_selector = LengthBasedExampleSelector(
  examples=examples, 
  example_prompt=example_prompt, 
  max_length=25,
)
example_selector

LengthBasedExampleSelector(examples=[{'input': '高', 'output': '矮'}, {'input': '胖', 'output': '瘦'}, {'input': '精力充沛', 'output': '萎靡不振'}, {'input': '快乐', 'output': '伤心'}, {'input': '黑', 'output': '白'}], example_prompt=PromptTemplate(input_variables=['input', 'output'], output_parser=None, partial_variables={}, template='\n词语:  {input}\n\n反义词:  {output}\n\n', template_format='f-string', validate_template=True), get_text_length=<function _get_length_based at 0x00000198314C45E0>, max_length=25, example_text_lengths=[10, 10, 10, 10, 10])

In [101]:
example_selector_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  example_separator="\n",
  prefix="我们来玩个反义词游戏，我说词语，你说它的反义词\n",
  suffix="现在轮到你了，词语: {input}\n反义词: ",
  input_variables=["input"],
)
example_selector_prompt.format(input="好")

'我们来玩个反义词游戏，我说词语，你说它的反义词\n\n\n词语:  高\n\n反义词:  矮\n\n\n\n词语:  胖\n\n反义词:  瘦\n\n\n现在轮到你了，词语: 好\n反义词: '

我们发现：examples列表中这些键值对并没有出现在提示词中：
{"input": "精力充沛", "output": "萎靡不振"},
{"input": "快乐", "output": "伤心"},
{"input": "黑", "output": "白"},

因为我们实例化这个少样本实例选择器的时候，选择的基于长度选择示例的 LengthBasedExampleSelector，并且实例化为一个对象，传参给了 FewShotPromptTemplate 的参数 example_selector， 并且设置了：max_length=25。如果示例的长度超过我们设置的最大长度，则会截断。

我们可以改一下最大长度的参数为100：max_length=100 。

In [104]:
example_selector = LengthBasedExampleSelector(
  examples=examples, 
  example_prompt=example_prompt, 
  max_length=100, # 
)
example_selector

LengthBasedExampleSelector(examples=[{'input': '高', 'output': '矮'}, {'input': '胖', 'output': '瘦'}, {'input': '精力充沛', 'output': '萎靡不振'}, {'input': '快乐', 'output': '伤心'}, {'input': '黑', 'output': '白'}], example_prompt=PromptTemplate(input_variables=['input', 'output'], output_parser=None, partial_variables={}, template='\n词语:  {input}\n\n反义词:  {output}\n\n', template_format='f-string', validate_template=True), get_text_length=<function _get_length_based at 0x00000198314C45E0>, max_length=100, example_text_lengths=[10, 10, 10, 10, 10])

所有的示例都被展示了。

[{'input': '高', 'output': '矮'}, {'input': '胖', 'output': '瘦'}, {'input': '精力充沛', 'output': '萎靡不振'}, {'input': '快乐', 'output': '伤心'}, {'input': '黑', 'output': '白'}]

## 其他示例选择器

在这些示例选择器中，实例化参数的确有所不同。它们都需要传入基础的参数，如examples和example_prompt，但根据选择器的不同，还有一些额外的参数需要设置。

对于LengthBasedExampleSelector，除了examples和example_prompt外，还需要传递max_length参数来设置示例的最大长度。

example_selector = LengthBasedExampleSelector(
    examples=examples, 
    example_prompt=example_prompt, 
    max_length=25,
)

对于MaxMarginalRelevanceExampleSelector，除了传入示例（examples）外，还需要传入一个用于生成语义相似性测量的嵌入类（OpenAIEmbeddings()），一个用于存储嵌入和执行相似性搜索的VectorStore类（FAISS），以及需要生成的示例数量（k=2）。

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=2,
)

对于NGramOverlapExampleSelector，除了examples和example_prompt外，还有一个threshold参数用于设定选择器的停止阈值。

example_selector = NGramOverlapExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    threshold=-1.0,
)

对于SemanticSimilarityExampleSelector，除了传入示例（examples）外，还需要传入一个用于生成语义相似性测量的嵌入类（OpenAIEmbeddings()），一个用于存储嵌入和执行相似性搜索的VectorStore类（Chroma），以及需要生成的示例数量（k=1）。

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    OpenAIEmbeddings(), 
    Chroma, 
    k=1
)

每种选择器都有其独特的参数设置，以满足不同的示例选择需求。